<p align="center"><img src="https://github.com/sotopia-lab/sotopia/blob/main/figs/title.png?raw=true" alt="drawing" width="100%"/></p>

# Sotopia Tutorial Series

<a href="https://colab.research.google.com/github/sotopia-lab/sotopia/blob/main/notebooks/tutorials/1.2-browse-data.ipynb" target="_parent"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"> </img> </a>

The following is the second tutorial of this series to teach how to quickly build your socially intelligent agents on the sotopia platform.

## Quick links

1. Basics
    
    1.1 [Playing with Sotopia in a Google Colab Notebook](https://colab.research.google.com/drive/14hJOfzpA37PRUzdlFgiqVzUGIhhngqnz?usp=sharing)
    
    1.2 [Browsing sotopia data](https://colab.research.google.com/drive/1Gi2U3cA0KG1nekw1A0N0DZpx6gKrj8Nc?usp=sharing)

2. Build your agents (coming soon!)
3. Evaluate your agents (coming soon!)
4. Extending Sotopia (coming soon!)

Please contact [Hao Zhu](https://zhuhao.me) for any questions!


# 1.2 Browsing [Sotopia](https://sotopia.world) data

Author: [Hao Zhu](https://zhuhao.me)

Running the code in this tutorial does not require familiarity of the redis setup. But if you have finished [Tutorial 1.1](https://colab.research.google.com/drive/14hJOfzpA37PRUzdlFgiqVzUGIhhngqnz?usp=sharing), you would have a better idea of the characters (in the format of `sotopia.database.AgentProfile`) and social tasks (in the format of `sotopia.database.EnvironmentProfile`). What you would be able to learn through this tutorial are:

1. How to load all of the existing data from sotopia(-pi) experiments, including 40 characters, hundreds of scenarios, and tens of thousands of interaction episodes.
2. How to browse them through indexing based on two awesome libraries `redis-om` and `pydantic`.

## Install Redis



In [ ]:
# @title Download Redis: This code block is exactly the same as [Tutorial 1.1](https://colab.research.google.com/drive/14hJOfzpA37PRUzdlFgiqVzUGIhhngqnz?usp=sharing).
%%capture
!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-7.2.0-v10.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
!tar -xvf redis-stack-server.tar.gz
# Installs libssl1.1 for Ubuntu 22 source: https://stackoverflow.com/questions/72133316/libssl-so-1-1-cannot-open-shared-object-file-no-such-file-or-directory
!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.22_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2.22_amd64.deb
%pip install redis

In [ ]:
%%capture
# @title Download existing data in a single file, and launch redis

!mkdir -p /content/redis-stack-server-7.2.0-v10/var/db/redis-stack
!curl -L https://huggingface.co/datasets/cmu-lti/sotopia-pi/resolve/main/dump.rdb?download=true --output /content/redis-stack-server-7.2.0-v10/var/db/redis-stack/dump.rdb
!./redis-stack-server-7.2.0-v10/bin/redis-stack-server --daemonize yes

If the above cell results in no error message, you have successfully set up your Redis server.

In [ ]:
# @title Your DB is now pre-populated with existing data:

import redis
from redis import BusyLoadingError
import time

client = redis.Redis(host="localhost", port=6379)
while True:
    try:
        client.ping()
        break
    except BusyLoadingError:
        time.sleep(1)
print(f"Voila, you have loaded {client.dbsize()} data!")

## Browse the data you have just loaded

In [ ]:
# @title First, install sotopia
%%capture
%pip install sotopia
import os

os.environ["REDIS_OM_URL"] = "redis://:@localhost:6379"

In [ ]:
# @title Show me a random character
import random
from rich import print

from sotopia.database import AgentProfile


all_character_pks = list(AgentProfile.all_pks())
pk = random.choice(all_character_pks)

print(AgentProfile.get(pk))

In [ ]:
# @title Show me a random social task
from sotopia.database import EnvironmentProfile

all_task_pks = list(EnvironmentProfile.all_pks())
pk = random.choice(all_task_pks)

print(EnvironmentProfile.get(pk))

In [ ]:
# @title Show me an Episode of the random social task before

from sotopia.database import EpisodeLog

all_episodes = list(EpisodeLog.find(EpisodeLog.environment == pk).all())
if len(all_episodes) == 0:
    print("No episodes found for this task, Please try to sample another task.")
else:
    episode = random.choice(all_episodes)
    print(episode)

### Find all young chefs in Sotopia

Powered by `redis_om`, the database of sotopia can be easily indexed by simple logics

In [ ]:
young_chefs = AgentProfile.find(
    (AgentProfile.occupation == "Chef") & (AgentProfile.age < 45)
).all()
print(
    f"There are {len(young_chefs)} young chefs in sotopia. They are: {', '.join(chef.first_name for chef in young_chefs)}"
)

### Tagging system in Sotopia

You might notice that Sotopia episodes are all "tagged" which allows for tracing the source of the generation. You can list all of the tags

In [ ]:
def get_episode_tag(episode_id):
    try:
        episode = EpisodeLog.get(episode_id)
        return episode.tag
    except Exception as _:
        return ""


set(get_episode_tag(episode) for episode in EpisodeLog.all_pks())

## Conclusion

In this very short tutorial, I demonstrated how to load the existing data from sotopia(-pi) experiments and browse the data with the easy-to-use api in sotopia. You can enumerate or find a random character and social task, and you can also index the interaction episodes with the task id or character id.